# 6. 쳇봇엔진만들기

## 6.1 쳇봇엔진구조

<h6 align="center">쳇봇 엔진의 핵심 기능</h6>

|핵심기능|설명|
|:------:|:----------------|
|질문의도분류|화자의 질문의도를 파악, 해당 질문을 의도분류모델을 이용해 의도클래스를 예측하는 문제|
|개체명 인식|화자의 질문에서 단어 토큰별 개체명을 인식. 이는 단어 토큰에 맞는 개체명을 예측하는 문제|
|핵심 키워드 추출|화자질문에서 핵심단어토큰을 추출. 형태소분석기로 핵심 키워드가 되는 명사,동사를 추출|
|답변 검색|해당질문의도, 개체명, 핵심키워드등을 기반으로 답변을 학습DB에서 검색|
|소켓 서버|다향한종류(카카오톡, 네이버톡톡)의 챗봇 클라이언트에서 요청 질문을 처리하기 위해 소켓서버|
||프로그램 역할을 한다. 따라서 이 책에서는 챗봇엔진 서버 프로그램이라 할 예정|

## 6.2 쳇봇엔진처리과정

1. 화자질의문장을 입력후 쳇봇엔진은 제일 먼저 전처리를 실행
1. 형태소분석기를 통해 토큰을 추출후 필요한 품사(명사, 동사 등)이외의 불용어를 제거
1. 의도분석과 개체명인식을 완료후 결과를 이용해서 적절한 답변을 학습된 DB에서 검색해서 화자에 답변을 전달

In [ ]:
%%writefile .\chatbot\utils\Preprocess.py
# Preprocess.py(1) - 전처리로직만 작성
from konlpy.tag import Komoran

class Preprocess:
    # 1. 생성자
    def __init__(self, userdic=None):
        # 1) 형태소분석기객체생성 및 초기화
        self.kormorn = Komoran(userdic=userdic)
        
        # 2) 불용어제거
        # 제외할 품사를 exclusion_tags 리스트에 정의
        # 참조 : https://docs.komoran.kr/firststep/postypes.html
        # 관계언, 기호, 어미, 접미사를 제거
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            'JX', 'JC', 'SF', 'SP', 'SS', 'SE', 'SO', 'EP', 
            'EF', 'EC', 'ETN', 'ETM', 'XSN', 'XSV', 'XSA']  
        

    # 2. 형태소분속기 pos 태거
    def pos(self, sentence):
        return self.kormorn.pos(sentence)
        
    # 3. 불용어제거후 필요한 품사정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list        

In [ ]:
# 전처리과정 테스트
from chatbot.utils.Preprocess import Preprocess

sentence = '내일 오전 10시에 탕수육을 주문하고 싶어'

# 1. 전처리객체생성
p = Preprocess(userdic='./chatbot/utils/user_dic.tsv')

# 2. 형태소분석
pos = p.pos(sentence)
print(pos)

# 3. 품사태그와 키워드를 출력
ret = p.get_keywords(pos, without_tag=False)
print(ret)

# 4. 품사태그없이 키워드를 출력
ret = p.get_keywords(pos, without_tag=True)
print(ret)


## 6.3 단어사전구축과 시퀀스 생성

* 말뭉치데이터(corpus.txt) -> train_toos/dict폴더에 저장
* 내일 -> 999, 오전 -> 111의 형태로 시퀀스생성

In [ ]:
%%writefile .\chatbot\train_tools\dict\create_dict.py
# create_dict.py(1) - 단어사전생성로직만 생성
from chatbot.utils.Preprocess import Preprocess
from tensorflow.keras import preprocessing
import pickle

# 1. 말뭉치데이터로딩함수 -> 말뭉치데이터를 list로 변환함수
def read_corpus_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]  # 헤더를 제거
    return data
    
# 2. 말뭉치데이터로딩
corpus_data = read_corpus_data('./chatbot/train_tools/dict/corpus.txt')

# 3. 말뭉치데이터에서 키워드만 추출 -> 사용자사전리스트를 생성
# corpus_data리스트에서 POS태깅후 단어리스트(dict)에 저장
p = Preprocess()
d = []
for c in corpus_data:
    pos = p.pos(c[1])
    for k in pos:
        d.append(k[0])

# 4. 사전에 사용될 index를 생성 -> 토크나이징처리를 해서 단어리스를 단어인덱스dict데이터를 생성
tokenizer = preprocessing.text.Tokenizer(oov_token='OOV')
tokenizer.fit_on_texts(d)
word_index = tokenizer.word_index
# len(word_index)

# 5. 사전파일생성
# 생성된 단어인덱스dict(word_index)객체를 파일로 저장
f = open('./chatbot/train_tools/dict/chatbot_dict.bin', 'wb')
try:
    pickle.dump(word_index, f)
except Exception as e:
    print(e)
finally:
    f.close()

In [ ]:
%%writefile .\chatbot\train_tools\dict\create_dict_test.py
# 단어사전테스트
import pickle
from chatbot.utils.Preprocess import Preprocess

# 1. 단어사전로딩
f = open('./chatbot/train_tools/dict/chatbot_dict.bin', 'rb')
word_index = pickle.load(f)
f.close()

# 2. 전처리객체생성
sentence = '내일 오전 10시에 탕수육을 주문하고 싶어 ㅋㅋ'
p = Preprocess(userdic='./chatbot/utils/user_dic.tsv')
pos = p.pos(sentence)

# 3. 테스트문장을 입력값으로 전달받아서 키워드와 인덱스를 출력
keywords = p.get_keywords(pos, without_tag=True)
for word in keywords:
    try:
        print(word, word_index[word])
    except KeyError:
        # 해당단어가 사전에 없을 때 OOV로 처리
        print(word, word_index['OOV'])

In [ ]:
!pip install JPype1
!pip show JPype1

In [ ]:
%%writefile .\chatbot\utils\Preprocess.py
# Preprocess.py(2) - 시퀀스생성로직 작성
# 단어인덱스 시퀀스변환 메서드를 추가
from konlpy.tag import Komoran
import pickle
import jpype  # JPype는 Python 으로 하여금 거의 모든 Java 라이브러리를 사용하게 한다.

class Preprocess:
    # 1. 생성자
    def __init__(self, word2index_dic='', userdic=None):
        # 0) 단어인덱스사전 로딩
        if(word2index_dic != ''):
            f = open(word2index_dic, 'rb')
            self.word_index = pickle.load(f)
            f.close()
        else:
            self.word_index = None
            
         # 1) 형태소분석기객체생성 및 초기화
        self.kormorn = Komoran(userdic=userdic)
        
        # 2) 불용어제거
        # 제외할 품사를 exclusion_tags 리스트에 정의
        # 참조 : https://docs.komoran.kr/firststep/postypes.html
        # 관계언, 기호, 어미, 접미사를 제거
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            'JX', 'JC', 'SF', 'SP', 'SS', 'SE', 'SO', 'EP', 
            'EF', 'EC', 'ETN', 'ETM', 'XSN', 'XSV', 'XSA']  
        
    # 2. 형태소분속기 pos 태거
    def pos(self, sentence):
        jpype.attachThreadToJVM() 
        return self.kormorn.pos(sentence)
        
    # 3. 불용어제거후 필요한 품사정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list     

    # 4. 키워드를 단어인덱스 시퀀스로 변환
    def get_wordidx_sequence(self, keywords):
        if self.word_index is None:
            return []
            
        w2i = []
        for word in keywords:
            try:
                w2i.append(self.word_index[word])
            except KeyError:
                w2i.append(self.word_index['OOV']) # 해당 단어가 사전에 없을 떄 OOV처리
                
        return w2i       

## 6.4 의도분류모델

* 쳇봇엔진에 화자의 질의가 입력되었을 때 전처리과정을 거친후 `화자의 문장의 의도를 분류`해야 한다.
* 클래스별로 분류하기위해서 `CNN모델을 사용`
* 실습하는 말뭉치의 의도는 5가지분류

<h6 align="center">쳇봇 엔진의 의도 분류 클래스 종류</h6>

|의도명|분류클래스|설명|
|:------:|:---:|:----------------|
|인사|0|텍스트가 인사말인 경우|
|욕설|1|텍스트가 욕설인 경우|
|주문|2|텍스트가 주문 관련 내용인 경우|
|예약|3|텍스트가 예약 관련 내용인 경우|
|기타|4|어떤 의도에도 포함되지 않는 경우|

In [ ]:
%%writefile .\chatbot\config\GlobalParams.py
# 글로벌 파라미터 정보 정의\
# 단어시퀀스의 벡터크기
MAX_SEQ_LEN = 15
def GlobalParams():
    global MAX_SEQ_LEN

### 6.4.1 의도분류모델학습
* 학습데이터 : ./chatbot/models/intent/total_train_data.csv
  - 음식주문과 예약을 위한 데이터셋

In [ ]:
# %%writefile .\chatbot\models\intent\train_model.py
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate


# 데이터 읽어오기
train_file = './chatbot/models/intent/total_train_data.csv'
data = pd.read_csv(train_file, delimiter=',')
queries = data['query'].tolist()
intents = data['intent'].tolist()

from chatbot.utils.Preprocess import Preprocess
p = Preprocess(word2index_dic="./chatbot/train_tools/dict/chatbot_dict.bin",
               userdic="./chatbot/utils/user_dic.tsv")

# 단어 시퀀스 생성
sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)


# 단어 인덱스 시퀀스 벡터 ○2
# 단어 시퀀스 벡터 크기
from chatbot.config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# (105658, 15)
print(padded_seqs.shape)
print(len(intents)) #105658

# 학습용, 검증용, 테스트용 데이터셋 생성 ○3
# 학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents))
ds = ds.shuffle(len(queries))

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수


# CNN 모델 정의  ○4
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(5, name='logits')(dropout_hidden)
predictions = Dense(5, activation='softmax')(logits)


# 모델 생성  ○5
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# 모델 학습 ○6
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

# 모델 평가(테스트 데이터 셋 이용) ○7
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

# 모델 저장  ○8
model.save('./chatbot/models/intent/intent_model.keras')

### 6.4.2 의도분류모델생성

In [ ]:
%%writefile .\chatbot\models\intent\IntentModel.py
# 쳇봇엔진 - 의도분류모델로딩(모델재사용)
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

# 의도분류모델모듈
class IntentModel:

    # 1) 생성자
    def __init__(self, model_name, preprocess):
        # 의도클래스레이블
        self.labels = {0:'인사', 1:'욕설', 2:'주문', 3:'예약', 4:'기타'}

        # 훈련된 의도분류모델 로딩
        self.model = load_model(model_name)

        # chatbot.Preprocess
        self.p = preprocess

    # 2) 의도클래스 예측함수
    def predict_class(self, query):
        # 1) 형태소분석
        pos = self.p.pos(query)

        # 2) 문장(query)내에서 키워드추출, 불용어제거
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 3) 벡터의 최대크기
        from chatbot.config.GlobalParams import MAX_SEQ_LEN
        
        # 4) 패딩처리
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

        # 5) 예측
        predict = self.model.predict(padded_seqs)
        predict_class = tf.math.argmax(predict, axis=1)

        # 6) 예측결과 즉, 의도클래스를 반환
        return predict_class.numpy()[0]

In [ ]:
%%writefile .\chatbot\models\intent\model_intent_test.py
# chatbot 엔진 - 의도분류모델을 테스트
from chatbot.utils.Preprocess import Preprocess
from chatbot.models.intent.IntentModel import IntentModel

p = Preprocess(word2index_dic='./chatbot/train_tools/dict/chatbot_dict.bin'
               , userdic='./chatbot/utils/user_dic.tsv')

intent = IntentModel(model_name='./chatbot/models/intent/intent_model.keras', preprocess=p)

query = '오늘 탕수육 주문 가능한가요?'
predict = intent.predict_class(query=query)
predict_label = intent.labels[predict]

print(query)
print(f'발화자의 질의를 예측한 의도의 클래스 = {predict}')
print(f'발화자의 질의를 예측한 의도의 레이블 = {predict_label}')

## 6.5 개체명인식모델학습

* 개체명인식모델을 `양방향LSTM모델을 사용`

<h6 align="center">개체명종류</h6>

|개체명|설명|
|:----:|:-----------------|
|B_FOOD|음식|
|B_DT, B_TI|날짜,시간(학습데이터의 영향으로 날짜와 시간을 혼용해서 사용)|
|B_PS|사람|
|B_OG|조직, 회사|
|B_LC|지역|

### 6.5.1 개체명인식모델 데이터셋

* 학습용데이터셋 : ./chatbot/models/ner/ner_train.txt

In [ ]:
# %%writefile .\chatbot\models\ner\train_model.py
# 챗봇엔진 - NER모델생성
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
from chatbot.utils.Preprocess import Preprocess

# 1. 학습파일로딩
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx+1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx-1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))

    return sents

p = Preprocess(word2index_dic='./chatbot/train_tools/dict/chatbot_dict.bin'
               , userdic='./chatbot/utils/user_dic.tsv')

# 2. 학습용말뭉치데이터 로딩
corpus = read_file('./chatbot/models/ner/ner_train.txt')

# 3. 말뭉치데이터에서 단어(2번째), BIO태그(4번째)만 로딩해서 학습용데이터셋을 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])

    sentences.append(sentence)
    tags.append(bio_tag)

print(f'샘플데이터셋의 크기 = \n {len(sentences)}')
print(f'0번째 샘플단어의 시퀀스 = \n {sentences[0]}')
print(f'0번째 샘플단어의 BIO태그 = \n {tags[0]}')
print(f'샘플단어의 시퀀스의 최대길이 = \n {max(len(l) for l in sentences)}')
print(f'샘플단어의 시퀀스의 평균길이 = \n {sum(map(len, sentences))/len(sentences)}')

# 4. 토크나이저 정의
# 단어시퀀스는 Preprocess객체에서 생성하기 때문에 BIO태그용 토크나이저 객체만 생성
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그정보는 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)

# 단어사전 및 태그사전의 크기
vocab_size = len(p.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print(f'BIO태그사전의 크기 = {tag_size}')
print(f'단어사전의 크기 = {vocab_size}')

# 5. 학습용 단어시퀀스 생성
# BIO태그는 토크나이저에서 생성된 사전데이터를 시퀀스번호형태로 인코딩한다.
X_train = [p.get_wordidx_sequence(sent) for sent in sentences]
y_train = tag_tokenizer.texts_to_sequences(tags)

index_to_ner = tag_tokenizer.index_word # 시퀀스인덱스를 NER로 변환하기위해 사용
index_to_ner[0] = 'PAD'

# 6. 시퀀스패딩처리
# 개체명인식모델의 입출력크기를 동일하게 설정하기 위해 시퀀스 패딩처리를 실행
# 벡터크기를 단어 시퀀스의 평균길이보다 여유있게 40으로 설정
max_len = 40
X_train = preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

# 7. 학습용 vs 검증용 = 8:2
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train
                                                    , test_size=.2, random_state=1234) 
X_train.shape, X_test.shape

# 8. 출력된 데이터를 one-hot encoding 처리
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)
print(f'학습용 샘플데이터셋 시퀀스 크기 = {X_train.shape}')
print(f'학습용 샘플데이터셋 레이블 크기 = {y_train.shape}')
print(f'검증용 샘플데이터셋 시퀀스 크기 = {X_test.shape}')
print(f'검증용 샘플데이터셋 레이블 크기 = {y_test.shape}')

# 9. 모델정의(Bi-LSTM)
# tag_size만큼 출력 뉴런에서 제일 확률은 출력값 1개를 선택하기 위해 softmax()함수 사용
# 손실함수는 categorical_crossentropy를 사용
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.5, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss="categorical_crossentropy", optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=128, epochs=10)

print(f'평가결과(정확도) = {model.evaluate(X_test, y_test)[1]}')

model.save('./chatbot/models/ner/ner_model.keras')

# 10. 시퀀스를 NER태그로 변환
# 예측값을 index_to_ner함수를 이용해서 태깅정보를 변환하는 함수 작성
def sequences_to_tag(sequences):
    result = []
    for sequence in sequences: # 전체시퀀스(sequences)애서 시퀀스를 하나씩 꺼내오기
        temp = []
        for pred in sequence:  # 시퀀스로 부터 예측값을 하나씩 꺼내오기
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace('PAD', 'O')) # 패딩처리된 타입 PAD를 기타(O)로 변경
        result.append(temp)

    return result

# 11. 테스트데이터셋의 NER예측 
# 1) fi_score를 계산하기 위해 import
#    predict()함수를 이용해서 f1-score값을 리턴
from seqeval.metrics import f1_score, classification_report

# 2) 테스트데이터셋으로 예측
#    X_test 데이터셋 시퀀스번호로 인코딩된 데이터셋(단어시퀀스, numpy배열)
#    테스트한 후 결과가 '예측된 NER태그정보가 저장된 numpy배열'을 리턴
y_predicted = model.predict(X_test)
pred_tags = sequences_to_tag(y_predicted)  # 예측된 개체인식명(NER)
test_tags = sequences_to_tag(y_test)       # 실제 개체인식명

# 3) f1_score 결과
#    classfication_report함수로 NER태그별로 계산된 정밀도, 재현율, f1_score를 출력
print(classification_report(test_tags, pred_tags))
print(f'f1-score = {f1_score(test_tags, pred_tags):.2%}')

### 6.5.2 개체명 인식 모듈 작성하기

In [ ]:
%%writefile .\chatbot\models\ner\NerModel.py
# 쳇봇엔진의 NER모델
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

# 개체명인식모델 모듈
class NerModel:

    # 1) 생성자
    def __init__(self, model_name, preprocess):
        
        # BIO태그클래스별 레이블정의
        self.index_to_ner = {1: 'O', 2: 'B_DT', 3: 'B_FOOD', 4: 'I', 5: 'B_OG', 
                     6: 'B_PS', 7: 'B_LC', 8: 'NNP', 9: 'B_TI', 0: 'PAD'} 
        # 의도분류모델을 로딩
        self.model = load_model(model_name)

        # 챗봇 Preprocess객체
        self.p = preprocess
    
    # 2) 개체명클래스예측메서드
    def predict(self, query):
        # 1) 형태소분석
        pos = self.p.pos(query)
        
        # 2) 문장내에서 키워드와 불용어제거
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]
        
        # 3) 패딩처리
        max_len = 40
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, padding='post'
                                                           , value=0, maxlen=max_len)
        predict = self.model.predict(np.array([padded_seqs[0]]))
        predict_class = tf.math.argmax(predict, axis=1)

        tags = [self.index_to_ner[i] for i in predict_class.numpy()[0]]

        return list(zip(keywords, tags))

    # 3) 예측된 개체명클래스를 태깅메서드
    def predict_tags(self, query):
        # 1) 형태소분석
        pos = self.p.pos(query)
        
        # 2) 문장내에서 키워드와 불용어제거
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 3) 패딩처리
        max_len = 40
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, padding='post'
                                                           , value=0, maxlen=max_len)
        predict = self.model.predict(np.array([padded_seqs[0]]))
        predict_class = tf.math.argmax(predict, axis=1)

        tags = []
        for tag_idx in predict_class.numpy()[0]:
            if tag_idx == 1: continue
            tags.append(self.index_to_ner[tag_idx])

        if len(tags) == 0: return None
        return tags

##### NerModel.py 사용

In [ ]:
%%writefile .\chatbot\models\ner\model_intent_test.py
# NerModel 모듈 사용(1)
from chatbot.utils.Preprocess import Preprocess
from chatbot.models.ner.NerModel import NerModel

p = Preprocess(word2index_dic='./chatbot/train_tools/dict/chatbot_dict.bin'
               , userdic='./chatbot/utils/user_dic.tsv')

ner = NerModel(model_name='./chatbot/models/ner/ner_model.keras', preprocess=p)
query = '오늘 오전 13시 10분에 탕수육을 주문하고 싶어요'
predicts = ner.predict(query)
print(predicts)

## 6.6 답변검색

* 발화자로 부터 입력된 문장을 전처리, 의도분류, 개체명인식과정을 통해 적절한 답변을 학습DB에서 검색
* 챗봇엔진이 자연어처리를 통해서 해석한 문장을 기초로 유사한 답변을 검색(DB에서 검색)
* 실제 상용화된 챗봇은 여건상 구현하기 힘들기 때문에 단순한 검색수준의 SQL을 이용한 DB기반으로 답변을 검색하는 방법을 구현

### 6.6.1 DB제어모듈생성

In [ ]:
%%writefile .\chatbot\utils\Database.py
# DB제어모듈
import pymysql
import pymysql.cursors
import logging

class Database:
    '''
        Chatbot의 Database 제어 모듈...
    '''
    def __init__(self, host, user, password, db_name, charset='utf8'):
        self.host = host
        self.user = user
        self.password = password
        self.db_name = db_name
        self.charset = charset
        self.conn = None
        
    def connect(self):
        if self.conn != None: return

        self.conn = pymysql.connect(
            host = self.host,
            user = self.user,
            password = self.password,
            db = self.db_name,
            charset = self.charset   
        )
        
    def close(self):
        if self.conn is None: return

        if not self.conn.open:
            self.conn = None
            return

        self.conn.close()
        self.conn = None

    # insert, delete, update
    def execute(self, sql):
        last_row_id = -1
        try:
            with self.conn.cursor() as cursor:
                cursor.execute(sql)
            self.conn.commit()
            last_row_id = cursor.lastrowid
            logging.debug("execute last_row_is : %d", last_row_id) 
        except Exception as e:
            logging.error(e)    

    def select_one(self, sql):
        result = None
        try:
            with self.conn.cursor(pymysql.cursors.DictCursor) as cursor:
                cursor.execute(sql)
                result = cursor.fetchone()
        except Exception as e:
           logging.error(e)
        finally:
            return result

    def select_all(self, sql):
        result = None
        try:
            with self.conn.cursor(pymysql.cursors.DictCursor) as cursor:
                cursor.execute(sql)
                result = cursor.fetchall()
        except Exception as e:
           logging.error(e)
        finally:
            return result        

### 6.6.2 답변검색모듈

In [1]:
%%writefile ./chatbot/utils/FindAnswer.py
# 답변검색모듈
class FindAnswer:

    def __init__(self, db):
        self.db = db

    # 1. 검색SQL문장작성
    def make_query(self, intent_name, ner_tags):
        sql = 'select * from chatbot_train_data'
        if intent_name != None and ner_tags == None:
            sql = sql + f" where intent = '{intent_name}'"
        elif intent_name != None and ner_tags != None:
            where = f" where intent = '{intent_name}'"
            if(len(ner_tags)>0):
                where += " and ("
                for ne in ner_tags:
                    where += f" ner like '%{ne}%' or"
                where = where[:-3] + ")"
            sql = sql + where

        # 동일한 답변이 2개이상인 경우, 랜덤으로 선택
        sql = sql + " order by rand() limit 1"

        return sql

    # 2. 답변검색
    # 의도명(intent_name)과 태그리스트(ner_tags)를 이용해서 질문의 답변을 검새
    def search(self, intent_name, ner_tags):

        # 1) 의도명, 개체인식명으로 답변검색
        sql = self.make_query(intent_name, ner_tags)
        answer = self.db.select_one(sql)
        
        # 2) 검색되는 답변이 없을 경우 의도명만 검색
        if answer is None:
            sql = self.make_query(intent_name, None)
            answer = self.db.select_one(sql)

        return (answer['answer'], answer['answer_image'])

    # 3. NER태그를 실제로 입력된 단어로 변환하는 함수
    # 질문 : 탕수육 대자로 한개 주문할게요 -> 개체명인식명 탕수육 B_FOOD로 처리
    # 답변 : {B_FOOD} 주문할게요	{B_FOOD} 주문 처리 완료되었습니다. 
    def tag_to_word(self, ner_predicts, answer):
        for word, tag in ner_predicts:
            # 변환해야하는 태그가 있는 경우 추가
            if tag=='B_FOOD' or tag=='B_DT' or tag=='B_TI':
                answer = answer.replace(tag, word) # {B_FOOD} -> {탕수육}
                
        answer = answer.replace('{', '')
        answer = answer.replace('}', '')
        return answer

Overwriting ./chatbot/utils/FindAnswer.py


### 6.6.3 챗봇엔진 동작 테스트

In [8]:
%%writefile ./chatbot/test/chabot_test.py
# chatbot엔진 동작 테스트하기
from chatbot.config.DatabaseConfig import *
from chatbot.utils.Database import Database
from chatbot.utils.Preprocess import Preprocess

# 1. 전처리객체생성
p = Preprocess(word2index_dic='./chatbot/train_tools/dict/chatbot_dict.bin'
               , userdic='./chatbot/utils/user_dic.tsv')

# 2. DB객체생성
db = Database(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db_name=DB_NAME)
db.connect()

# 3. 발화자질의
query = '오전에 탕수육 10개를 주문합니다'
# query = '오전에 탕수육  주문합니다'
# query = '화자의 질문의도를 파악합니다.'
# query = '안녕하세요'
# query = '자장면 주문할게요'

# 4. 발화자의도파악
from chatbot.models.intent.IntentModel import IntentModel
intent = IntentModel(model_name='./chatbot/models/intent/intent_model.keras', preprocess=p)
predict = intent.predict_class(query)
intent_name = intent.labels[predict]

# 5. 개체명인식
from chatbot.models.ner.NerModel import NerModel
ner = NerModel(model_name='./chatbot/models/ner/ner_model.keras', preprocess=p)
predicts = ner.predict(query)
ner_tags = ner.predict_tags(query)

# 6. 출력확인
print(f'발화자의 질의 = {predict}')
print(f'발화자의 의도 = {intent_name}')
print(f'발화자의 질의의 개체명 = {predicts}')
print(f'발화자의 질의의 NER태그(답변검색에 필요한 NER태그) = {ner_tags}')

# 7. 답변검색
from chatbot.utils.FindAnswer import FindAnswer

try:
    f = FindAnswer(db)
    # print(f.make_query(intent_name, ner_tags))
    answer_text, answer_image = f.search(intent_name, ner_tags)
    answer = f.tag_to_word(predicts, answer_text)
except Exception as e:
    print(e)
    answer = "죄송합니다. 무슨 말인지 모르겠어요!"

print(f'답변검색결과 = {answer}')

db.close()

Writing ./chatbot/test/chabot_test.py


## 6.7 챗봇엔진서버

* 챗봇API인 카카카오톡이나 네이버톡톡같은 메신저 플랫폼을 이용
* 실습은 카카오톡 API를 사용

### 6.7.1 통신프로토콜정의

* 서버와 클라이언트간 JSON형태로 통신

```json
# 질의 텍스트
{
    "Query": "자장면 주문할게요",
    "BotType": "Kakao"
}

# 답변
{
    "Query": "자장면 주문할게요",
    "Intent": "주문",
    "NER":"[('자장면', 'B_FOOD'), ('주문', 'O)]",
    "Answer":"자장면 주문 처리 감사",
    "AnswerImageUrl":""
}
```

### 6.7.2 챗봇서버모듈

In [4]:
%%writefile ./chatbot/utils/BotServer.py
# 챗봇서버모듈
import socket

class BotServer:
    # 1. chatbot의 서버포트번화 동시접속자수를 정의
    def __init__(self, srv_port, listen_num):
        self.port = srv_port # 서버접속포트번호
        self.listen = listen_num
        self.mySock = None
        
    # 2. socket생성
    #    파이썬에서 지원하는 저수준 네트워킹인터페이스 API를 사용하기 쉽게 작성된 랩퍼함수
    #    TCP/IP 소켓생성후 접속자수 만큼 클라이언트의 연결을 수락
    def create_socket(self):
        # self.mySock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.mySock = socket.socket()
        self.mySock.bind(("0.0.0.0", int(self.port)))
        self.mySock.listen(int(self.listen))
        return self.mySock
        
    # 3. 클라이언트 대기후 연결을 수락하는 메서드
    #    연결요청시 클라이언트와 통신가능한 소켓객체를 리턴
    #    반환값(conn, address)을 tuple형태로 리턴
    def ready_for_client(self):
        return self.mySock.accept()
        
    # 4. socket반환
    def get_socket(self):
        return self.mySock

Overwriting ./chatbot/utils/BotServer.py


### 6.7.3 챗봇서버 메인 프로그램
* 실행방법 : python[.exe] bot.py

In [5]:
%%writefile ./bot.py
import threading
import json

from chatbot.config.DatabaseConfig import *
from chatbot.utils.Database import Database
from chatbot.utils.BotServer import BotServer
from chatbot.utils.Preprocess import Preprocess
from chatbot.models.intent.IntentModel import IntentModel
from chatbot.models.ner.NerModel import NerModel
from chatbot.utils.FindAnswer import FindAnswer

# 1. 전처리
p = Preprocess(word2index_dic='./chatbot/train_tools/dict/chatbot_dict.bin'
               , userdic='./chatbot/utils/user_dic.tsv')

# 2. 의도파악학습모델
intent = IntentModel(model_name='./chatbot/models/intent/intent_model.keras', preprocess=p)

# 3. 개체인식학습모델
ner = NerModel(model_name='./chatbot/models/ner/ner_model.keras', preprocess=p)

# 4. 클라이언트가 연결되는 순간 실행되는 Thread함수
#    적절한 답변을 검색한후에 요청(requext)한 클라이언트에 응답(response)
def to_client(conn, addr, params):
    db = params['db']

    try:
        db.connect()

        # 1) data 수신(발화자의 질의)
        # conn은 쳇봇클라이언트의 소캣객체, recv()메서드는 데이터의
        # 수신이 완료될 때까지 블럭킹, 최대 2048bytes만큼 데이터를 수신
        # 에러(연결중단 or 예외)가 있을 경우에 recv()메서드는 None을 리턴
        read = conn.recv(2048)  # 수신데이터가 있을 때 까지 블럭킹
        print('='*60)
        print(f'Connection from : {str(addr)}') # localhost:5000?....

        if read is None or not read: # 클라이언트연결중단 or 에러가 있을 겨우
            print('Client Connection Stop!!')
            exit(0)
        
        # 2) json형태로 변환
        recv_json_data = json.loads(read.decode()) 
        print(f'데이터수신 : {recv_json_data}')
        query = recv_json_data['Query']
        
        # 3) 발화자의 의도파악
        intent_predict = intent.predict_class(query)
        intent_name = intent.labels[intent_predict]
    
        # 4) 개체명 인식
        ner_predicts = ner.predict(query)
        ner_tags = ner.predict_tags(query)

        # 5) DB에서 답변검색
        try:
            f = FindAnswer(db)
            answer_text, answer_image = f.search(intent_name, ner_tags)
            answer = f.tag_to_word(ner_predicts, answer_text)
        except Exception as e:
            print(e)
            answer = "죄송합니다. 무슨 말인지 모르겠어요! 조금 더 학습을 할게요 ㅠㅠ"
            answer_image = None

        send_json_data_str = {
            "Query": query,
            "Answer": answer,
            "AnswerImageUrl": answer_image,
            "Intent": intent_name,
            "NER": ner_tags   
        }

        message = json.dumps(send_json_data_str)
        conn.send(message.encode())
        
    except Exception as e:
        print(e)
    finally:
        if db is not None:
            db.close()

# 5. chatbot application start
if __name__ == '__main__':
    
    # 1) 답변검색을 위한 DB 연결
    db = Database(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db_name=DB_NAME)
    print('DB Connection Successful!!')

    port = 5000
    listen = 100

    # 2) 쳇봇서버동작 - 챗봇클라이언트 연결을 대기(무한 loop)
    bot = BotServer(port, listen)
    bot.create_socket()
    print('ChatBot Server Start!!')

    while True:
        conn, addr = bot.ready_for_client()
        params = {
            'db': db
        }

        client = threading.Thread(target=to_client, args=(conn, addr, params))
        client.start()

Overwriting ./bot.py


### 6.7.4 챗봇 클라이언트 프로그램
* cd ./lec/05.python
* cmd창 : python ./chatbot/test/chatbot_client_test.py

In [6]:
%%writefile ./chatbot/test/chatbot_client_test.py
# chatbot 클라이언트 테스트 프로그램
import socket
import json

# 1. 쳇봇엔진서버접속정보
host = "127.0.0.1"
port = 5000

# 2. 클라이언트프로그램 Start
while True:
    query = input('질문을 입력하세요(작업종료는 q) => ') # 발화자의 질의
    print(f'발화자질문 : {query}')
    if(query=='q'): exit(0)

    print('='*60)
    mySocket = socket.socket()
    mySocket.connect((host, port))

    # 1) 챗봇엔진에 질의 요청
    json_data = {
        "Query": query,  
        "BotType": "myBotService"
    }
    message = json.dumps(json_data)
    mySocket.send(message.encode())

    # 2) 쳇본엔진에 답변출력
    data = mySocket.recv(2048).decode()
    ret_data = json.loads(data)
    print(f"답변 = {ret_data['Answer']}")
    print(type(ret_data), ret_data)

    # 3) 챗봇서버에 연결될 소켓 해제
    mySocket.close()

Overwriting ./chatbot/test/chatbot_client_test.py


### 6.7.5 챗봇 애플리케이션 시작

* cmd창(Termnal)
  1. 챗봇서버 시작       : root폴더 >python bot.py
  2. 챗봇클라이언트 시작 : root폴더 > python ./chatbot/test/chatbot_client_test.py
* 주의할 점
  - 서버가 비정상적으로 종료가 될 때 사용중인 port(5000)는 계속해서 활성화 상태일 수 있다.
  - 이때, 활성화된 process id를 확인 후에 해당 작업을 kill을 시켜야 한다.
  - `netstat -ano | findstr 5000`명령으로 process를 찾은 후에
  - `taskkill /f /pid 검색된process-id` 명령을 실행

## 6.8 맺음말

1. 실제 음식주문용 챗봇을 사용하려면 발화자의 요청이 2개 이상의 B_FOOD를 인식해야 하고
2. 음식주문수량을 확인할 수 있는 개체명이 추가 되어야 한다.
3. 현재, 실습한 음식주문챗봇은 `자장면 1개, 탕수육 대2개 주문할게요`와 같은 주문은 정확하게 처리할 수 없다.
4. 정확한 주문을 처리하는 챗봇을 만들이 위해서 학습데이터와 개체명인식데이터가 필요하다.
5. 이와 같이 딥러닝모델에서는 학습용데이터가 매우 중요하다.
6. 우리가 목표로 하는 시스템에 맞는 `데이터수집, 정제하는데 대부분의 시간이 필요`하다.